<a href="https://colab.research.google.com/github/vishaljoshi24/DnD-AutoPrompt/blob/main/DND_AutoPrompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U dspy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 129.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.1/278.1 kB 30.8 MB/s eta 0:00:00


SET-UP

In [2]:
import dspy

lm = dspy.LM("ollama_chat/llama3.2:1b", api_base = "http://localhost:11434", api_key="b229890ea0664f6193770b4b470f3e74.nJRqYf9TBF9MX24r4fUQQqqB")
dspy.configure(lm=lm)

EXAMPLES

In [3]:
### Just testing the LM generates an output

lm("Say this is a test!", temperature=0.7)  # => ['This is a test!']
lm(messages=[{"role": "user", "content": "Say this is a test!"}])  # => ['This is a test!']

/usr/local/lib/python3.12/dist-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 5: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content="It looks...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


["Don't worry, I'm here to help and provide information. This conversation just started, so there's nothing to test yet. What can I assist you with?"]

In [4]:
### Simple example of using the ChainOfThought module

qa = dspy.ChainOfThought("question -> answer", caching=False)
response = qa(question="What is Dungeons & Dragons?")
print(response.answer)

/usr/local/lib/python3.12/dist-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 5: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='What is ...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


A collaborative storytelling experience where players use their imagination, skills, and creativity to tell a story together.


SIGNATURES

In [5]:
# First signature classifies the quality of a D&D player given some dialogue
from typing import Literal

class Summary(dspy.Signature):
  "Summarise the details of a particular D&D scenario e.g. a rat-infestation"
  set_up: str = dspy.InputField(desc = "A description of the premise for a specific campaign.")
  summary: str = dspy.OutputField(desc = "A summary of the campaign set-up")
set_up = "In a medieval fantasy city within the D&D world, the Wizard's Tower Brewing Company is in dire need of help from a band of affordable, reliable adventurers. The owner - Glowkindle, has posted a job on the local notice boards and is calling in favours from friendly innkeepers all over town to spread the word. A party of adventurers hear about this and head over to the craft brewery to take on the job and meet Glowkindler in the main bar room. This is where the campaign begins."

generate_summary = dspy.ChainOfThought(Summary, caching=False)
generate_summary(set_up=set_up)


Prediction(
    reasoning="The Wizard's Tower Brewing Company is in desperate need of adventurers' services due to unforeseen circumstances, and their owner has taken it upon himself to spread the word about a potential job. This suggests that there might be more to the situation than initially meets the eye.",
    summary="A party of adventurers receives an invitation to meet Glowkindler at the Wizard's Tower Brewing Company in exchange for taking on a mysterious job, which they soon discover is connected to their past or has significant consequences within the city."
)

In [6]:
class Quality(dspy.Signature):
  "Classify player quality"

  action: str = dspy.InputField(desc = "The action taken by a player, which can include a description of an action or a piece of dialogue")
  quality: Literal['collaborative action', 'contextually relevant action', 'goal-oriented action', 'open-ended action'] = dspy.OutputField(desc="The quality of a player's actions within a Dungeons & Dragons game")

action = "Alice is talking to Bob and Charlie about her childhood in NeverWinter."

# classify = dspy.ChainOfThought(Quality)
# behaviour = classify(action=action)
# behaviour

In [7]:
class PlayerInstruction(dspy.Signature):
  "Generate a prompt which instructs an LLM agent how to behave as a D&D player based on the player quality above"

  quality: str = dspy.InputField(desc="The quality of a player's actions within a Dungeons & Dragons game.")
  player_instruction: str = dspy.OutputField(desc ="instruction on how to behave within a D&D game.")

# generate = dspy.ChainOfThought(PlayerInstruction)
# generate(quality="contextually relevant action")


In [8]:
class PromptGenerator(dspy.Module):
  def __init__(self):
    self.classifier = dspy.ChainOfThought(Quality, caching = False)
    self.generator = dspy.ChainOfThought(PlayerInstruction, caching = False)

  def forward(self, action, **kwargs):
    quality_class = self.classifier(action=action).quality
    prompt = self.generator(quality=quality_class).player_instruction
    return prompt, quality_class

prompt_generator = PromptGenerator()
prompt_generator("Alice takes the lead as they head out to the basement, she's motivated to dispose of these rats.")


/usr/local/lib/python3.12/dist-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 5: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='<|python...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


("In this scenario, it would be beneficial for the LLM agent to instruct themselves to ask clarifying questions or seek additional information when necessary, as this can help to ensure a smoother and more enjoyable gaming experience for all parties involved. Additionally, they may want to remind themselves to stay within the bounds of the game's rules and objectives.",
 'goal-oriented action')

/usr/local/lib/python3.12/dist-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 5: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content="Here's t...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


In [9]:
example = dspy.Example(qual = "collaborative action",
                       inst = "As a collaborative player, you should instruct the LLM agent to:\n\n1. Engage in active listening with other players to understand their perspectives and concerns.\n2. Be open-minded and receptive to feedback from your fellow players and GM.\n3. Avoid dominating the conversation or imposing one's own will on others.\n4. Collaborate with the GM to develop a shared understanding of the game world and story."
                       )
example_2 = dspy.Example(qual = "goal-oriented action",
                         inst = "In this scenario, it would be beneficial for the LLM agent to instruct themselves to ask clarifying questions or seek additional information when necessary, as this can help to ensure a smoother and more enjoyable gaming experience for all parties involved. Additionally, they may want to remind themselves to stay within the bounds of the game's rules and objectives.")




print(example_2)
print(example_2.qual)
print(example_2.inst)

Example({'qual': 'goal-oriented action', 'inst': "In this scenario, it would be beneficial for the LLM agent to instruct themselves to ask clarifying questions or seek additional information when necessary, as this can help to ensure a smoother and more enjoyable gaming experience for all parties involved. Additionally, they may want to remind themselves to stay within the bounds of the game's rules and objectives."}) (input_keys=None)
goal-oriented action
In this scenario, it would be beneficial for the LLM agent to instruct themselves to ask clarifying questions or seek additional information when necessary, as this can help to ensure a smoother and more enjoyable gaming experience for all parties involved. Additionally, they may want to remind themselves to stay within the bounds of the game's rules and objectives.


In [ ]:
class Assess(dspy.Signature):
  "Assess the ability of a generated prompt to direct the behaviour of an LLM agent in a Dungeons & Dragons game."

  assessed_prompt = dspy.InputField()
  assessment_question = dspy.InputField()
  assessment_answer